# Métodos no supervisados y detección de anomalías: tarea de asignación (semana 2)
En esta semana de curso se ha profundizado en la resolución del problema de agrupamiento con métodos particionales, así como la validación de sus resultados. En esta tarea de asignación tendrás que implementar algunas de las medidas de validación interna y externa explicadas en la lección 2. Además, podrás comprobar que el resultado es correcto utilizando las implementaciones disponibles en la librería [*sklearn*](https://scikit-learn.org/stable/modules/clustering.html#clustering-evaluation).

## Descripción de la tarea
La tarea consta de dos apartados:
1. Cálculo de medidas de validación externa: Codifica funciones que permitan calcular la medida *Purity* y los índices de *Rand* y *Fowlkes-Mallows*.
2. Cálculo de medidas de validación interna: Codifica funciones que permitan calcular las medidas *WSS*, *BSS* y *Silhouette coefficient*.

## Instrucciones
A lo largo del *notebook* encontrarás varios apartados con el comentario **COMPLETAR**. Añade el código necesario para realizar lo que se pide en dicho apartado. Es recomendable utilizar la función *print* para visualizar el resultado tras invocar a las funciones que calculen las medidas.

## 1. Medidas de validación interna
Paso 1.1: Importa los paquetes necesarios

In [92]:
import scipy.spatial.distance as dist
import numpy as np

Paso 1.2: Define una función para calcular la medida ***Purity***. Esta función necesita dos parámetros:
*   particion_real: vector que contiene, para cada instancia de un hipotético conjunto de datos, el índice del grupo al que pertenece.
*   particion_obtenida: vector que contiene, para cada instancia del mismo conjunto de datos, el índice del grupo al que ha sido asignado.



In [93]:
def purity(particion_real:int, particion_obtenida:int):
  # COMPLETAR

  n_clus = {}

  # Inicializamos el diccionario con los distintos grupos
  for pt_o in np.unique(particion_obtenida):

    n_clus[str(pt_o)] = []

  # Creamos agrupaciones
  for pt_o, pt_r in zip(particion_obtenida, particion_real):

    n_clus[str(pt_o)].append(pt_r)

  numerator = 0

  # Seleccionamos el elemento más común del grupo y contamos el número de ocurrencias
  for key in n_clus.keys():

    most_common = max(set(n_clus[key]), key=n_clus[key].count)

    numerator += n_clus[key].count(most_common)

  return numerator / len(particion_obtenida)


Paso 1.3: Declara dos vectores para reflejar la asignación real de etiquetas (partición real) y una hipotética agrupación (partición). Utilizamos el ejemplo de la lección 2, con 7 instancias y 2 grupos (etiquetas) reales.

In [94]:
particion_real = [1, 2, 1, 2, 2, 1, 2]
particion_obtenida = [1, 2, 1, 1, 2, 3, 3]

Paso 1.4: Invoca a la función *purity* con los dos vectores anteriores como parámetros e imprime el resultado (debe ser el mismo que el del ejemplo de la lección, esto es, 0.7143 aproximadamente).

In [95]:
res_purity = purity(particion_real, particion_obtenida)
print(res_purity)

0.7142857142857143


Paso 1.5: Define una función para calcular el **Índice de Rand**. La función debe tener dos parámetros, la partición real y la obtenida.

In [96]:
from scipy.special import comb

def rand_index(particion_real:int, particion_obtenida:int):
  # COMPLETAR

  tp_fp = comb(np.bincount(particion_real), 2).sum()
  
  tp_fn = comb(np.bincount(particion_obtenida), 2).sum()

  A = np.c_[(particion_real, particion_obtenida)]

  tp = sum(comb(np.bincount(A[A[:, 0] == i, 1]), 2).sum() for i in set(particion_real))

  fp = tp_fp - tp

  fn = tp_fn - tp

  tn = comb(len(A), 2) - tp - fp - fn

  return (tp + tn) / (tp + fp + fn + tn)

Paso 1.6: Invoca a la función *rand_index* con los vectores declarados en el apartado 1.3, e imprime el resultado.

In [97]:
res_rand = rand_index(particion_real, particion_obtenida)
print(res_rand)

0.5238095238095238


Paso 1.7: Comprueba que el resultado obtenido es igual al que se obtiene si se utiliza la implementación de *sklearn*.

In [98]:
# COMPLETAR
from sklearn.metrics.cluster import rand_score

print(rand_score(particion_real, particion_obtenida))

0.5238095238095238


Paso 1.8: Define una función para calcular el **Índice de Fowlkes-Mallows**. La función debe tener como parámetros un vector con la partición real y otro con la partición de referencia.

In [99]:
def fowlkes_mallows(particion_real:int, particion_obtenida:int):
  # COMPLETAR

  tp_fp = comb(np.bincount(particion_real), 2).sum()
  
  tp_fn = comb(np.bincount(particion_obtenida), 2).sum()

  A = np.c_[(particion_real, particion_obtenida)]

  tp = sum(comb(np.bincount(A[A[:, 0] == i, 1]), 2).sum() for i in set(particion_real))

  fp = tp_fp - tp

  fn = tp_fn - tp

  return np.sqrt((tp / (tp + fn)) * (tp / (tp + fp)))

Paso 1.9: Invoca a la función con los vectores definidos en el apartado 1.3.

In [100]:
res_fw = fowlkes_mallows(particion_real, particion_obtenida)
print(res_fw)

0.29814239699997197


Paso 1.10: Comprueba que el resultado obtenido es el mismo que al utilizar la implementación disponible en *sklearn*.

In [101]:
# COMPLETAR
from sklearn.metrics.cluster import fowlkes_mallows_score

fowlkes_mallows_score(particion_real, particion_obtenida)

0.29814239699997197

## 2. Medidas de validación externa

Paso 2.1: Para facilitar el cálculo de medidas de validación externa, implementa las siguientes funciones auxiliares:

*   *suma_cuadrado*: Suma de las distancias al cuadrado entre un par de puntos de tipo *float*
*   *calculo_centroide*: Dada una lista de puntos (de tipo *float*), devuelve las coordenadas del centroide (media en cada dimensión).



In [102]:
def suma_cuadrado(x:float, y:float):
  # COMPLETAR

  return np.sqrt((x[0] - y[0]) ** 2 + (x[1] - y[1]) ** 2)
  

def calcular_centroide(puntos:list):
  # COMPLETAR

  x, y = zip(*puntos)

  return [sum(x) / len(puntos), sum(y) / len(puntos)]

Paso 2.2: Define una función para calcular la medida **WSS** (*within cluster sum of squares*), la cual debe recibir un conjunto de datos de n dimensiones (en forma de lista) y una asignación de grupos para dicho conjunto. 

In [103]:
def wss(datos:list, particion:int):
  # COMPLETAR
  wss_sum = 0

  groups = np.unique(particion)

  for grp in groups:

    # Recogemos los indices de los puntos de cada grupo
    points_grp = [i for i,x in enumerate(particion) if x == grp]

    # Seleccionamos los puntos
    data_grp = list(np.array(datos)[points_grp])

    # Calculamos el centroide del grupo
    centroid = calcular_centroide(data_grp)

    # Calculamos la distancia de cada punto del grupo a su centroide
    for cl in data_grp:
      
      wss_sum += suma_cuadrado(cl, centroid)
  
  return wss_sum

Paso 2.3: Crea un conjunto de datos de dos dimensiones y una partición con tres grupos

In [104]:
datos = list()
np.random.seed(0)
tam = 10
for i in range(0, tam):
  x = np.random.randint(0, 10)
  y = np.random.randint(0, 10)
  datos.append(np.array([x,y]))
print(datos)
particion = [1, 1, 2, 3, 2, 1, 2, 3, 3, 2]


[array([5, 0]), array([3, 3]), array([7, 9]), array([3, 5]), array([2, 4]), array([7, 6]), array([8, 8]), array([1, 6]), array([7, 7]), array([8, 1])]


Paso 2.4: Invoca a la función *wss* para calcular la cohesión de la partición.

In [105]:
res_wss = wss(datos, particion)
print(res_wss)

31.920505779961665


Paso 2.5: Define una función para calcular la medida **BSS** (*between cluster sum of squares*), la cual debe recibir un conjunto de datos de n dimensiones (en forma de lista) y una asignación de grupos para dicho conjunto.

In [106]:
def bss(datos:list, particion:int):
  # COMPLETAR
  
  bss_sum = 0

  groups = np.unique(particion)

  # Calculamos punto medio del conjunto de datos
  m_centroid = calcular_centroide(datos)

  for grp in groups:

    # Recogemos los indices de los puntos de cada grupo
    points_grp = [i for i,x in enumerate(particion) if x == grp]

    # Seleccionamos los puntos
    data_grp = list(np.array(datos)[points_grp])

    # Calculamos el centroide del grupo
    i_centroid = calcular_centroide(data_grp)

    # Calculamos distanca del centroide del grupo al centroide del conjunto de datos, 
    # multiplicándolo por el tamaño del grupo
    bss_sum += len(data_grp) * suma_cuadrado(m_centroid, i_centroid)

  return bss_sum

Paso 2.6: Utilizando el mismo conjunto de datos y partición del apartado 2.3, invoca a la función *wss* e imprime el resultado.

In [107]:
res_bss = bss(datos, particion)
print(res_bss)


16.316670078708334


Paso 2.7: Para facilitar la implementación de una función que calcule el **Silhoutte coefficient**, define primero la siguiente función:

*   distancia_punto_grupo: Función que calcula la distancia media de un punto, dado sus coordenadas, a todos los puntos incluidos en una lista. Puedes utilizar una implementación externa de la distancia euclídea (por ejemplo, la de *scipy*) para calcular la distancia entre cada par de puntos.

In [108]:
def distancia_punto_grupo(punto:float, grupo:list):
  # COMPLETAR
  
  distances = []

  for pnt in grupo:

    distances.append(dist.euclidean(punto, pnt))

  return np.mean(distances)

Paso 2.8: Implementa un función que calcule el **Silhoutte coefficient** para un único punto. Sus parámetros son el punto para el que se calcula el valor, el conjunto de datos completo (en forma de lista), la asignación de grupos a puntos y el índice del punto en el conjunto de datos.

In [109]:
def silhouette_punto(punto:float, datos:list, particion:int, indice:int):
  # COMPLETAR

  # Calculamos a

  # Recogemos los indices de los puntos del mismo cluster que el punto dado
  points_grp = [i for i,x in enumerate(particion) if x == particion[indice]]

  # Seleccionamos los puntos
  data_grp = list(np.array(datos)[points_grp])

  # Eliminamos del grupo el punto
  data_grp = [x for x in data_grp if not (x == punto).all()]

  # Calculamos la distancia del punto a todos los puntos del mismo grupo
  a = distancia_punto_grupo(punto, data_grp)

  # Calculamos b

  # Calculamos la distancia al resto de grupos
  groups = list(np.unique(particion))
  
  groups.remove(particion[indice])

  b = []  

  for grp in groups:

    points_not_grp = [i for i,x in enumerate(particion) if x == grp]

    # Seleccionamos los puntos no pertenecientes al grupo del punto
    data_not_grp = list(np.array(datos)[points_not_grp])

    # Insertamos la distancia del punto al grupo que no pertenece
    b.append(distancia_punto_grupo(punto, data_not_grp))

  # # Devolvemos el valor usando el valor mínimo de b, ya que es la distancia al grupo más cercano
  return (min(b)-a) / max(a, min(b))

Paso 2.9: Implementa una función que calcule el **Silhouette coefficient** para una partición, invocando para ello a la función definida en el apartado 2.8.

In [110]:
def silhoutte_particion(datos:list, particion:int):
  # COMPLETAR

  sil_score = []

  for i in range(len(datos)):

    sil_score.append(silhouette_punto(datos[i], datos, particion, i))
  
  return np.mean(sil_score)

Paso 2.10: Invoca a la función que calcula el coeficiente de silueta para el ejemplo del apartado 2.3 y comprueba que el resultado coincide con el que proporciona *sklearn*.

In [111]:
# COMPLETAR

from sklearn.metrics import silhouette_score

print(silhoutte_particion(datos, particion))

print(silhouette_score(datos, particion))

-0.12527718550104788
-0.12527718550104788
